In [2]:
from openai import OpenAI
client = OpenAI()

upload training file

In [ ]:
training_file = client.files.create(
    # upload training file
    file = open("", "rb"),
    purpose = "fine-tune"
)
training_file_id = training_file.id
print("Training file id : ", training_file_id)

create fine-tune job
edit suffix to help you find out the fune-tuned model

In [ ]:
job = client.fine_tuning.jobs.create(
    training_file = training_file_id, 
    model="gpt-3.5-turbo",
    # suffix help you find out the fine-tuned model
    suffix = "fine-tune"
)
job_id = job.id
print("Job ID : ", job_id)

check job status, fine-tuned model, and job list

In [ ]:
# Retrieve the state of a fine-tune & get fine-tuned model ID
print(client.fine_tuning.jobs.retrieve(job_id))
print("Job status :", client.fine_tuning.jobs.retrieve(job_id).status)
fine_tuned_model = client.fine_tuning.jobs.retrieve(job_id).fine_tuned_model
print("fine-tuned model :", fine_tuned_model)

client.fine_tuning.jobs.list(limit=10)

# Cancel a job
# client.fine_tuning.jobs.cancel("ftjob-abc123")

# Delete a fine-tuned model (must be an owner of the org the model was created in)
# client.models.delete("ft:gpt-3.5-turbo:acemeco:suffix:abc123")

create Result.xlsx

In [12]:
import openpyxl

# open the test file
workbook = openpyxl.load_workbook('Train_Test.xlsx')
sheet = workbook['Test']

sheet['E1'] = "Response"

for row in range(2, 98):
    prompt = [{"role":"user", "content": sheet["A" + str(row)].value}]
    response = client.chat.completions.create(
        model = fine_tuned_model,
        messages = prompt
    )
    sheet["E" + str(row)].value = response.choices[0].message.content

workbook.save("Result.xlsx")